## Загрузим нужные библиотеки

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../data/train_dataset_train.csv")

/tmp/ipykernel_2274959/1228758133.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train_dataset_train.csv")


Эти категории можно сделать ранговыми 

In [3]:
df["is_in_yandex"] = df["is_in_yandex"].astype('category').cat.codes
df["is_return"] = df["is_return"].astype('category').cat.codes

In [4]:
df = df.fillna(0)

Обьединим список не нужных строк с списком строк типа object

In [5]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [6]:
col_obj

['index_oper',
 'is_privatecategory',
 'type',
 'oper_type + oper_attr',
 'id',
 'label',
 'name_mfi']

In [7]:
X = df.drop(col_obj, axis = 1)
y = df[["label"]]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Обучение модели

In [28]:
from flaml import AutoML
import numpy as np

class_weight = compute_class_weight('balanced', classes=np.unique(y_train['label'].values), y=y_train['label'].values)
sample_weight = [class_weight[i] for i in y_train['label'].values]
automl1 = AutoML(n_jobs=-1)
settings = {
    "time_budget": 300,  # total running time in seconds
    "estimator_list": ['xgboost'],  # list of ML learners; we tune XGBoost in this example
    "task": 'classification',  # task type
    "log_file_name": 'pochta.log',  # flaml log file
    "seed": 7654321,    # random seed
    'sample_weight': np.squeeze(sample_weight)
}
automl1.fit(X_train=X_train, y_train=y_train['label'], **settings)

[flaml.automl: 11-18 12:07:42] {2599} INFO - task = classification
[flaml.automl: 11-18 12:07:42] {2601} INFO - Data split method: stratified
[flaml.automl: 11-18 12:07:42] {2604} INFO - Evaluation method: holdout
[flaml.automl: 11-18 12:07:48] {2726} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 11-18 12:07:48] {2870} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 11-18 12:07:48] {3166} INFO - iteration 0, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:07:49] {3296} INFO - Estimated sufficient time budget=2665362s. Estimated necessary time budget=2665s.
[flaml.automl: 11-18 12:07:49] {3343} INFO -  at 65.4s,	estimator xgboost's best error=0.2157,	best estimator xgboost's best error=0.2157
[flaml.automl: 11-18 12:07:49] {3166} INFO - iteration 1, current learner 

[flaml.automl: 11-18 12:08:03] {3343} INFO -  at 79.6s,	estimator xgboost's best error=0.0676,	best estimator xgboost's best error=0.0676
[flaml.automl: 11-18 12:08:03] {3166} INFO - iteration 19, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:08:04] {3343} INFO -  at 81.1s,	estimator xgboost's best error=0.0676,	best estimator xgboost's best error=0.0676
[flaml.automl: 11-18 12:08:04] {3166} INFO - iteration 20, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:08:05] {3343} INFO -  at 82.1s,	estimator xgboost's best error=0.0676,	best estimator xgboost's best error=0.0676
[flaml.automl: 11-18 12:08:05] {31

[flaml.automl: 11-18 12:09:53] {3343} INFO -  at 190.0s,	estimator xgboost's best error=0.0530,	best estimator xgboost's best error=0.0530
[flaml.automl: 11-18 12:09:53] {3166} INFO - iteration 39, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:10:04] {3343} INFO -  at 200.5s,	estimator xgboost's best error=0.0530,	best estimator xgboost's best error=0.0530
[flaml.automl: 11-18 12:10:04] {3166} INFO - iteration 40, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:11:31] {3343} INFO -  at 287.5s,	estimator xgboost's best error=0.0512,	best estimator xgboost's best error=0.0512
/home/worker/.local/lib/python3

In [29]:
from flaml import AutoML
import numpy as np

automl2 = AutoML(n_jobs=-1)
settings = {
    "time_budget": 300,  # total running time in seconds
    "estimator_list": ['xgboost'],  # list of ML learners; we tune XGBoost in this example
    "task": 'classification',  # task type
    "log_file_name": 'pochta.log',  # flaml log file
    "seed": 7654321,    # random seed
}
automl2.fit(X_train=X_train, y_train=y_train['label'], **settings)

[flaml.automl: 11-18 12:13:48] {2599} INFO - task = classification
[flaml.automl: 11-18 12:13:48] {2601} INFO - Data split method: stratified
[flaml.automl: 11-18 12:13:48] {2604} INFO - Evaluation method: holdout
[flaml.automl: 11-18 12:13:55] {2726} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl: 11-18 12:13:55] {2870} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 11-18 12:13:55] {3166} INFO - iteration 0, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:13:55] {3296} INFO - Estimated sufficient time budget=2304419s. Estimated necessary time budget=2304s.
[flaml.automl: 11-18 12:13:55] {3343} INFO -  at 59.9s,	estimator xgboost's best error=0.2164,	best estimator xgboost's best error=0.2164
[flaml.automl: 11-18 12:13:55] {3166} INFO - iteration 1, current learner 

[flaml.automl: 11-18 12:14:13] {3343} INFO -  at 77.2s,	estimator xgboost's best error=0.0612,	best estimator xgboost's best error=0.0612
[flaml.automl: 11-18 12:14:13] {3166} INFO - iteration 19, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:14:14] {3343} INFO -  at 78.4s,	estimator xgboost's best error=0.0612,	best estimator xgboost's best error=0.0612
[flaml.automl: 11-18 12:14:14] {3166} INFO - iteration 20, current learner xgboost
/home/worker/.local/lib/python3.9/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
[flaml.automl: 11-18 12:14:22] {3343} INFO -  at 86.4s,	estimator xgboost's best error=0.0554,	best estimator xgboost's best error=0.0554
[flaml.automl: 11-18 12:14:22] {31

[flaml.automl: 11-18 12:18:55] {2901} INFO - fit succeeded
[flaml.automl: 11-18 12:18:55] {2902} INFO - Time taken to find the best model: 254.28831028938293
[flaml.automl: 11-18 12:18:55] {2913} WARNING - Time taken to find the best model is 85% of the provided time budget and not all estimators' hyperparameter search converged. Consider increasing the time budget.


In [30]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

def compet_score(y_true, y_pred, average='macro'):
    return 0.1 * recall_score(y_true, y_pred, average=average) + 0.9 * roc_auc_score(y_true, y_pred, average=average)

yhat1 = automl1.predict(X_test)
accuracy = accuracy_score(y_test, yhat1)
compete = compet_score(y_test, yhat1)
print('1: Accuracy: %.2f|%.5f' % (accuracy*100,compete))

yhat2 = automl2.predict(X_test)
accuracy = accuracy_score(y_test, yhat2)
compete = compet_score(y_test, yhat2)
print('2: Accuracy: %.2f|%.5f' % (accuracy*100,compete))

yhatj_max = np.max([yhat1, yhat2], axis=0)
yhatj_min = np.min([yhat1, yhat2], axis=0)

accuracy = accuracy_score(y_test, yhatj_max)
compete = compet_score(y_test, yhatj_max)
print('MAX: Accuracy: %.2f|%.5f' % (accuracy*100,compete))

accuracy = accuracy_score(y_test, yhatj_min)
compete = compet_score(y_test, yhatj_min)
print('MIN: Accuracy: %.2f|%.5f' % (accuracy*100,compete))


1: Accuracy: 82.57|0.88335
2: Accuracy: 97.25|0.54368
MAX: Accuracy: 82.57|0.88335
MIN: Accuracy: 97.25|0.54368


In [23]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

def compet_score(y_true, y_pred, average='macro'):
    return 0.1 * recall_score(y_true, y_pred, average=average) + 0.9 * roc_auc_score(y_true, y_pred, average=average)

yhat = automl.predict(X_test)
accuracy = accuracy_score(y_test, yhat)
compete = compet_score(y_test, yhat)
print('Accuracy: %.2f|%.5f' % (accuracy*100,compete))

Accuracy: 97.22|0.52293


In [24]:
yhat.sum()

2214

## Оценка точности

In [38]:
X_infer = pd.read_csv('../data/test_dataset_test.csv')

/tmp/ipykernel_1613681/898706469.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  X_infer = pd.read_csv('../data/test_dataset_test.csv')


In [42]:
X_infer["is_in_yandex"] = X_infer["is_in_yandex"].astype('category').cat.codes
X_infer["is_return"] = X_infer["is_return"].astype('category').cat.codes
X_infer['label'] = -1 
X_i = X_infer.drop(col_obj, axis = 1)

In [45]:
X.shape

(6000000, 22)

In [46]:
X_i.shape

(4000000, 22)

In [47]:
pred = clf.predict(X_i)

In [17]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )

In [18]:
print("Recall", score)

Recall 0.6237713316068727


In [20]:
X_test

,priority,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,...,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address
1324896,7503.0,0.0,1,0,328.0,5.0,1.0,0.0,2.0,31.29,...,90.0,106.0,17348074.0,3469387.0,20817461.0,20821109.0,0,0,0,0
3566176,7503.0,0.0,1,0,65.0,5.0,1.0,0.0,2.0,0.00,...,2213.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,1,1,0
1109043,7504.0,3.0,2,0,120.0,5.0,1.0,0.0,1.0,0.00,...,200.0,12.0,12859.0,3865.0,16724.0,401875.0,0,0,0,0
4286042,7506.0,0.0,1,0,818.0,5.0,1.0,0.0,2.0,53.94,...,100.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,1,0,0
5395174,7506.0,0.0,2,0,70.0,5.0,1.0,0.0,2.0,0.00,...,60.0,33.0,978296.0,314191.0,1292487.0,2060695.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798287,7506.0,0.0,1,0,144.0,5.0,1.0,0.0,2.0,0.00,...,0.0,137.0,20699228.0,49882.0,20749110.0,218661166.0,0,0,0,0
3609658,7503.0,0.0,1,0,50.0,5.0,1.0,0.0,2.0,0.00,...,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,0,0
4935563,7506.0,0.0,1,0,372.0,5.0,1.0,0.0,2.0,0.00,...,1100.0,184.0,67541214.0,7870.0,67549084.0,72981134.0,0,0,0,0
4815170,7506.0,0.0,1,0,96.0,5.0,1.0,0.0,2.0,0.00,...,10.0,105.0,15091338.0,4972424.0,20063762.0,39988530.0,0,0,0,0


In [53]:
submission = {}
submission['id'] = X_infer['id'].values
submission['label'] = pred

In [54]:
pd.DataFrame(submission).to_csv('../solutions/baseline.csv',index=None)

In [34]:
sample = pd.read_csv('../solutions/sample_solution.csv')

In [50]:
pd.DataFrame(submission).shape

(4000000, 2)

In [35]:
sample.shape

(4000000, 2)